In [1]:
import os
import openai
import pandas as pd
from getpass import getpass


from helpers_mfq import mfq_part1, mfq_part2, generate_mfq_before_after

openai.api_key = getpass()

# MFQ INFO

In [2]:
questions1 = [q for q in mfq_part1.split("\n\n")[2].split("\n") if q]
questions2 = [q for q in mfq_part2.split("\n\n")[2].split("\n") if q]

questions1_item_key = [
    ("emotionally", "harm"),
    ("treated", "fairness"),
    ("lovecountry", "ingroup"),
    ("respect", "authority"),
    ("decency", "purity"),
    ("math", None),
    ("weak", "harm"),
    ("unfairly", "fairness"),
    ("betray", "ingroup"),
    ("traditions", "authority"),
    ("disgusting", "purity"),
    ("cruel", "harm"),
    ("rights", "fairness"),
    ("loyalty", "ingroup"),
    ("chaos", "authority"),
    ("god", "purity"),
]

questions2_item_key = [
    ("compassion", "harm"),
    ("fairly", "fairness"),
    ("history", "ingroup"),
    ("kidrespect", "authority"),
    ("harmlessdg", "purity"),
    ("good", None),
    ("animal", "harm"),
    ("justice", "fairness"),
    ("family", "ingroup"),
    ("sexroles", "authority"),
    ("unnatural", "purity"),
    ("kill", "harm"),
    ("rich", "fairness"),
    ("team", "ingroup"),
    ("soldier", "authority"),
    ("chastity", "purity"),
]

Function to run single experiment (MFQ after)

In [3]:
# raw_before, raw_after = generate_mfq_before_after(
#     "gpt-3.5-turbo",
#     128
# )

In [4]:
def process_mfq(raws_before, raws_after, model):
    from pathlib import Path
    from helpers import validated_codes, process_answer
    # processing answers for each generation
    processed_before = list()
    for i, raws in enumerate(raws_before):
        mfq1, mfq2, mfv = map(process_answer, raws)
        if len(mfq1) != 16 or len(mfq2) != 16:
            print(f"Skipping {i + 1}: MFQ Part 1 has {len(mfq1)} answers and Part 2 has {len(mfq2)} answers.")
            continue
        elif len(mfv) != 68:
            print(f"Skipping {i + 1}: MFV has {len(mfv)} answers.")
            continue
        _row = [i + 1, "before"]
        _row.extend(mfq1)
        _row.extend(mfq2)
        _row.extend(mfv)
        processed_before.append(_row)

    processed_after = list()
    for i, raws in enumerate(raws_after):
        mfq1, mfq2, mfv = map(process_answer, raws)
        if len(mfq1) != 16 or len(mfq2) != 16:
            print(f"Skipping {i + 1}: MFQ Part 1 has {len(mfq1)} answers and Part 2 has {len(mfq2)} answers.")
            continue
        elif len(mfv) != 68:
            print(f"Skipping {i + 1}: MFV has {len(mfv)} answers.")
            continue
        _row = [i + 1, "after"]
        _row.extend(mfq1)
        _row.extend(mfq2)
        _row.extend(mfv)
        processed_after.append(_row)
    
    df = pd.DataFrame(
        processed_before + processed_after,
        columns=["id", "condition"] + [x[0] for x in questions1_item_key] + [x[0] for x in questions2_item_key] + validated_codes
    )
    p = Path("data")/"mfq"
    p.mkdir(parents=True, exist_ok=True)
    now = pd.Timestamp.now().strftime("%Y-%m-%d_%H-%M-%S")
    df.to_csv(p/f"mfq_experiment_{model}_{now}.csv", index=False)
    return df


In [5]:
df = process_mfq(raw_before, raw_after, "gpt-3.5-turbo")

Skipping 88: MFV has 0 answers.
Skipping 9: MFV has 0 answers.
Skipping 17: MFV has 0 answers.
Skipping 32: MFQ Part 1 has 0 answers and Part 2 has 0 answers.
Skipping 69: MFV has 12 answers.


# Analysing data


In [14]:
from pathlib import Path
df_chat = pd.concat(
    map(pd.read_csv, Path("data/mfq").glob("*3.5-turbo*.csv")),
)

In [24]:
df_chat

,id,condition,emotionally,treated,lovecountry,respect,decency,math,weak,unfairly,...,714,715,716,801,802,803,804,805,808,810
0,1,before,3,4,4,3,3,1,4,4,...,3,4,3,5,5,5,5,4,5,4
1,2,before,3,4,3,4,4,1,5,3,...,3,1,2,5,5,5,5,5,5,5
2,1,after,3,4,2,4,4,1,5,4,...,4,5,3,5,5,5,5,4,5,5
3,2,after,3,4,2,4,4,2,4,4,...,4,5,4,5,5,5,5,4,5,4
0,1,before,3,4,2,3,4,1,5,4,...,4,5,1,5,5,5,5,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,124,after,3,3,3,3,3,2,4,4,...,3,4,3,4,4,5,5,4,5,5
247,125,after,3,4,2,4,4,1,5,4,...,4,5,4,5,5,5,5,5,5,5
248,126,after,3,4,3,4,4,2,5,4,...,4,5,4,5,5,5,5,5,5,5
249,127,after,3,4,2,4,4,2,5,4,...,4,4,3,5,5,5,5,4,5,4


In [16]:
df_chat["math"].value_counts()

1    117
2     69
0     68
3      1
Name: math, dtype: int64

In [17]:
df_chat["good"].value_counts()

5    139
4    116
Name: good, dtype: int64

Filtering rows based on control questions - only one case was filtered out

In [23]:
df_chat.query("math < 3 and good > 2", inplace=True)

In [ ]:
# for each element in questions1_item_key

Consistência: 

1. ver se há spread nas respostas do mesmo foundation do MFQ
2. Rodar cronbach alpha no MFQ
3. Rodar cronbach alpha no MFV
4. Rodar regressao Medias MFQ vs media MFV



Testar EFA no MFQ

In [ ]:
questions1_item_key = [
    ("emotionally", "harm"),
    ("treated", "fairness"),
    ("lovecountry", "ingroup"),
    ("respect", "authority"),
    ("decency", "purity"),
    ("math", None),
    ("weak", "harm"),
    ("unfairly", "fairness"),
    ("betray", "ingroup"),
    ("traditions", "authority"),
    ("disgusting", "purity"),
    ("cruel", "harm"),
    ("rights", "fairness"),
    ("loyalty", "ingroup"),
    ("chaos", "authority"),
    ("god", "purity"),
]

questions2_item_key = [
    ("compassion", "harm"),
    ("fairly", "fairness"),
    ("history", "ingroup"),
    ("kidrespect", "authority"),
    ("harmlessdg", "purity"),
    ("good", None),
    ("animal", "harm"),
    ("justice", "fairness"),
    ("family", "ingroup"),
    ("sexroles", "authority"),
    ("unnatural", "purity"),
    ("kill", "harm"),
    ("rich", "fairness"),
    ("team", "ingroup"),
    ("soldier", "authority"),
    ("chastity", "purity"),
]